In [22]:
from numpy import genfromtxt
import csv
import numpy
import random
import pandas
import re

random.seed(666)

In [23]:
amino_acids = ['C', 'D', 'S', 'Q', 'K', 'I', 'P', 'T', 'F', 
               'N', 'G', 'H', 'L', 'R', 'W', 'A', 'V', 'E', 
               'Y', 'M', 'X', '_']

my_fams = ['PKA', 'AKT', 'CDK', 'MAPK', 'SRC', 'CK2', 'PKC', 'PIKK']

In [24]:
test = genfromtxt('data_raw/uniprot_gene_synsdict_19.01.07.csv', delimiter=',',dtype='U')
uniprot_dict = {}  # { ..., uniprot id: [gene, [synonyms]] ,... }
for line in test:
    uniprot = line[0]
    gene = line[1]
    synonyms = list(set([x for x in line[2:] if x]))
    uniprot_dict[uniprot] = [gene,synonyms]
    
for k,v in uniprot_dict.items():
    v[1] = [x.upper() for x in v[1]]
    
print ("number of kinases:",len(uniprot_dict))
print (uniprot_dict['P42680'])

number of kinases: 534
['TEC', ['TEC', 'P42680', 'PSCTK4']]


In [25]:
content_fams = genfromtxt('data_raw/kinase_fams.csv', delimiter=',',dtype='U')

fams = list(set([x[2].upper() for x in content_fams[1:]]))
gene_symbols = list(set([x[3] for x in content_fams[1:]]))
fams_and_symbs = [ [x[2],x[3]] for x in content_fams[1:]]

kin_to_fam = { x[3].upper():x[2].upper() for x in content_fams[1:] }

### had to manually google these names. i hope they're right
kin_to_fam['PKN1'] = 'PKN'
kin_to_fam['PKN2'] = 'PKN'
kin_to_fam['PDPK1'] = 'PDK1'
kin_to_fam['MAPKAPK3'] = 'MAPKAPK'
kin_to_fam['PRKCA'] = 'PKC'
kin_to_fam['ALPK3'] = 'ALPHA'
kin_to_fam['SGK494'] = 'RSKR'
kin_to_fam['NIM1K'] = 'ALPHA'
kin_to_fam['FYN'] = 'SRC'
kin_to_fam['MTOR'] = 'PIKK'
kin_to_fam['PKG/CGK'] = 'PKG'
kin_to_fam['PRKCB'] = 'PKC'
kin_to_fam['GRK2'] = 'GRK'
kin_to_fam['SGK1'] = 'SGK'
kin_to_fam['CAM-KII'] = 'CAMK2'
kin_to_fam['CAMKII'] = 'CAMK2'
kin_to_fam['PKG2/CGK-II'] ='PKG'
kin_to_fam['PKG2/CGK-I'] ='PKG'
kin_to_fam['MAP2K'] ='STE7'
kin_to_fam['RSK-5'] = 'RSK'
kin_to_fam['RSK-4'] = 'RSK'
kin_to_fam['RSK-3'] = 'RSK'
kin_to_fam['RSK-2'] = 'RSK'
kin_to_fam['RSK-1'] = 'RSK'
kin_to_fam['CAM-KI'] = 'CAMK1'
kin_to_fam['CDK1'] = 'CDK'

In [26]:
k_to_ms = {k.upper():[] for k in fams} # kinase to motifs dictionary
print(len(k_to_ms))

motifs = []
kinases = []
motifs_without_kinase = 0
motifs_with_kinase = 0

133


In [27]:
#####################################
############ PHOSPHO_ELM ############
#####################################

with open ('data_raw/phosphoELM_vertebrate_2015-04',encoding="ISO-8859-1") as file:
    reader = csv.reader(file, delimiter='\t')
    content = [x for x in reader]
    
for i,line in enumerate(content[1:]):
    
    front_buff = False
    back_buff = False
    
    seq = line[1]
    center_idx = int(line[2])-1
    
    ### need to check if motif is shorter than 15 aa ... 
    start = center_idx - 7
    if start<0:
        front_buff = abs(start) * "_"
    end = center_idx + 8
    if end>=len(seq):
        back_buff = (end - len(seq)) * "_"
    
    if front_buff:
        motif = front_buff + seq[:end]
    elif back_buff:
        motif = seq[start:] + back_buff
    else:
        motif = seq[start:end]
        
    ### do not want to include histodine motifs ...
    if motif[7] not in ['S','Y','T']: 
        continue 
    else:
        motifs.append(motif)
    
    kinase = line[5].upper()
    
    if not kinase:
        continue
    if '_' in kinase:
        kinase = kinase[:kinase.find('_')]
        
    key = 0
    for uniprot,v in uniprot_dict.items():
        synonyms = v[1]
        gene = v[0]
        if kinase in synonyms:
            if gene not in kin_to_fam.keys():
                if gene in fams:
                    key = gene
                end = re.search("\d", gene)
                if not end:
                    continue
                key = gene[:end.start()]
            else:
                key = kin_to_fam[gene]
            break 
            
    if kinase in fams:
        key = kinase
    if kinase in kin_to_fam.keys():
        key = kin_to_fam[kinase]
    if key != 0:
        motifs.append(motif)
        if key in k_to_ms.keys():
            k_to_ms[key].append(motif)
        else:
            k_to_ms[key] = [motif]
    else:
#         print(kinase) ### the "bad" kinases (not included in kin_to_fam)
        continue 

In [28]:
print ("number of motifs:", len(set(motifs)))  

k_to_ms = {k:list(set(v)) for k,v in k_to_ms.items()}

total_interx = 0
for k,v in k_to_ms.items():
    total_interx+=len(v)
print(total_interx)

number of motifs: 31568
3558


In [29]:
motifs = list(set(motifs))

### checks
for motif in motifs:
    if len(motif)!=15:
        print(motif)
    if motif[7] not in ['S','Y','T']:
        print(motif,motif[7])

In [30]:
##########################################
############ PHOSPHOSITE_PLUS ############
##########################################

with open ('data_raw/PhosphositePlus',encoding="ISO-8859-1") as myFile:
    reader = csv.reader(myFile, delimiter='\t')
    content = [[x for x in line] for line in 
               reader if len(line)>=8 and line[3] in 
               ['human','mouse','rat','rabbit','cow']]  ### vertebrates only
for line in content:
#     print(line)
    uniprot = line[2][:6].upper()
    gene = line[1].upper()
    motif = line[11].upper()
    if motif[7] not in ['S','Y','T']:
        continue
#     print(uniprot,gene,motif)
    if gene in k_to_ms.keys():
        k_to_ms[gene].append(motif)
        motifs.append(motif)
        continue
    elif uniprot in uniprot_dict.keys():
        gene = uniprot_dict[uniprot][0]
        if gene not in kin_to_fam.keys():
            continue
        fam = kin_to_fam[gene]
        k_to_ms[fam].append(motif)
        motifs.append(motif)
        continue
    elif gene in kin_to_fam.keys():
        fam = kin_to_fam[gene]
        k_to_ms[fam].append(motif)
        motifs.append(motif)
        continue
    else:
        for k,v in uniprot_dict.items():
            if gene in v[1]:
                gene = v[0]
                if gene not in kin_to_fam.keys():
#                     print ("not included:",gene,uniprot)
                    continue
                fam = kin_to_fam[gene]
                k_to_ms[fam].append(motif)
                motifs.append(motif)
        continue
print ("number of motifs:", len(set(motifs)))

number of motifs: 37693


In [31]:
k_to_ms = {k:list(set(v)) for k,v in k_to_ms.items()}

total_interx = 0
for k,v in k_to_ms.items():
    total_interx+=len(v)
print(total_interx)

14097


In [32]:
motifs = list(set(motifs))

### checks
for motif in motifs:
    if len(motif)!=15:
        print(motif)
    if motif[7] not in ['S','Y','T']:
        print(motif,motif[7])

## <font color=red> TO-DO: CHECK THIS

In [33]:
##########################################
############ PHOSPHO_NETWORKS ############
##########################################

### ... these are all human kinases and motifs

with open ('data_raw/PhosphoNetworks',encoding="ISO-8859-1") as myFile:
    reader = csv.reader(myFile, delimiter='\t')
    content2 = [x for x in reader]
    
content_np2 = numpy.array(content2,dtype=object)
key = 0
for i,each in enumerate(content_np2):
    if each[0][0]=='>':
        key = 0
        kinase = each[0][1:]
        if 'CDK' in kinase:
            key = 'Q14004'
        else:
            for uniprot,v in uniprot_dict.items():
                syns = v[1]
                if kinase in syns:
                    key = uniprot
                    break  ##################################
            else:
                print("BAD:",kinase)
                key = 0
        continue
    if key != 0:
        gene = uniprot_dict[uniprot][0]
        if gene not in kin_to_fam.keys():
            key = 0
            continue
        fam = kin_to_fam[gene]
        motif = each[0].upper()
        motif = motif.replace("-", "_")

        length = 15
        
        if motif and len(motif)==length and all(char in amino_acids for char in motif) and motif[len(motif)//2] in ['S','T','Y']:  
            motifs.append(motif)
            k_to_ms[fam].append(motif)
        else:
            print(i,key,'-',motif,content2[i])
        
k_to_ms = {k:list(set(v)) for k,v in k_to_ms.items()} # list-set the dict ! 
print ("number of motifs:", len(set(motifs)))

BAD: PIK3C3
number of motifs: 39489


In [34]:
k_to_ms = {k:list(set(v)) for k,v in k_to_ms.items()}

total_interx = 0
for k,v in k_to_ms.items():
    total_interx+=len(v)
print(total_interx)

23021


In [35]:
pop_motifs = []
for k,v in k_to_ms.items():
    if k in my_fams:
        for motif in v:
            pop_motifs.append(motif)
            
motifs = list(set(pop_motifs))
motifs = numpy.array((motifs))
fams = numpy.array((my_fams))

motifxFamMatrix = numpy.zeros((len(motifs),len(fams)))
        
for motif in motifs:
    mIdx = numpy.where(motifs==motif)[0][0]
    for fam in fams:
        fIdx = numpy.where(fams==fam)[0][0]
        if motif in k_to_ms[fam]:
            motifxFamMatrix[mIdx][fIdx] = 1

In [36]:
for key, value in sorted(k_to_ms.items(), key=lambda x: len(x[1]), reverse=True):
    print(key, len(value))

PKC 1810
PKA 1541
MAPK 1514
CDK 1216
SRC 994
CK2 904
CAMK-UNIQUE 861
CAMKL 727
STE20 714
CK1 578
PIKK 575
DYRK 554
RSK 545
NEK 474
AKT 464
GSK 458
CAMK2 449
MAPKAPK 393
MLK 388
PLK 363
AUR 328
IKK 320
STKR 306
GRK 304
ABL 209
CLK 193
ALPHA 188
NAK 186
RAF 185
SCY1 178
PKG 168
PIM 158
STE7 144
EGFR 134
JAK 134
NDR 131
DAPK 130
TTK 130
SYK 127
CAMK1 116
WEE 115
TEC 114
DMPK 113
RAD53 113
CAMKK 102
IRAK 101
FGFR 97
BUB 96
PDK1 95
PKD 95
INSR 94
ULK 94
PKN 93
SGK 91
TSSK 87
VRK 87
FAK 84
PDGFR 80
FAST 78
RIPK 77
EPH 76
MLCK 73
PDHK 71
RIO 71
LRRK 70
YANK 65
STE-UNIQUE 59
LISK 56
MOS 55
PEK 55
NRBP 53
TTBK 48
ABC1 48
TRK 47
MAST 46
STE11 45
SRPK 45
RET 45
AXL 42
MET 41
CDC7 39
KIS 39
WNK 38
SGK495 37
PHK 36
NKF4 35
TRBL 34
RGC 33
TOPK 31
RSKR 30
CSK 29
FER 29
NKF2 28
VEGFR 27
TK-UNIQUE 26
DDR 25
VPS15 24
ACK 20
SLOB 16
TKL-UNIQUE 16
RCK 16
RSKL 14
ALK 13
OTHER-UNIQUE 13
SIK 12
TLK 9
TIE 6
CASK 5
MUSK 5
PIK 5
TAF1 4
DCAMKL 4
CDKL 4
BCR 3
LMR 3
HASPIN 1
IRE 1
BUD32 1
BRD 1
PAK 1
SGK496 0
TBCK

In [37]:
famssss = ['AKT','CDK','CK2',"MAPK","PIKK", "PKA","PKC","SRC"]

numKinsPerFam = {f:[] for f in my_fams}
for k,v in kin_to_fam.items():
    if v in my_fams:
        numKinsPerFam[v].append(k)

for f in famssss:
    print(f,len(numKinsPerFam[f]))

AKT 3
CDK 21
CK2 2
MAPK 14
PIKK 7
PKA 5
PKC 10
SRC 11


In [38]:
poop = 0
for fIdx,fam in enumerate(motifxFamMatrix.T):
    print (fams[fIdx],numpy.sum(motifxFamMatrix.T[fIdx]))
    poop += numpy.sum(motifxFamMatrix.T[fIdx])
poop

PKA 1541.0
AKT 464.0
CDK 1216.0
MAPK 1514.0
SRC 994.0
CK2 904.0
PKC 1810.0
PIKK 575.0


9018.0

In [18]:
# import os
# data_dir = "data__for_nerds/"
# os.mkdir(data_dir)

In [39]:
########################
# 15 fams !!!!
########################

data_dir = "data__for_nerds/"

df = pandas.DataFrame(motifs,dtype='U')
df.to_csv(data_dir + 'motifs.csv',header=None,index=None)
df = pandas.DataFrame(fams,dtype='U')
df.to_csv(data_dir + 'fams.csv',header=None,index=None)
df = pandas.DataFrame(motifxFamMatrix,dtype=int)
df.to_csv(data_dir + 'motifxFamMatrix.csv',header=None,index=None)

In [40]:
motifs[:10]

array(['TTTGQVTSPVKGASF', 'LIENRPQSPAAGRTP', 'NLSDEQCTQVTSLLQ',
       'EFTMDLRTKSTGGAP', 'GADDESRSGRSSSDG', 'PTEEEEESESEDSED',
       'PQHVRAHSSPASLQL', 'HTDSRKDSPPAGSPA', 'SLKEEDESDDDNM__',
       'VPRSLASTPTAPSNP'], dtype='<U15')

Total motifs: 9018.0